In [35]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

In [40]:
moving_window_size = 3
input_size = moving_window_size
hidden_size = 3
num_layers = 1
LEARNING_RATE = 0.1

#Generatig some sample data:
x = np.array([0.1*i for i in range(10)])
y = 2*x + 1
print('X=\n',x)
print('y=\n',y)

X=
 [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
y=
 [1.  1.2 1.4 1.6 1.8 2.  2.2 2.4 2.6 2.8]


**Creating sequences from data**

Given past 3 points predict the next future point

In [38]:
#Creating a sequence of data for LSTM
def moving_window_sampler(x, y, window_size):
    x_train, y_train = [], []
    for i in range(len(x) - window_size):
        x_train.append(x[i:i + window_size])
        y_train.append(y[i + window_size])
    return x_train, y_train

x_train, y_train = moving_window_sampler(x, y, window_size=moving_window_size)
print('X_train sequence:\n')
print(x_train)
print('\nY_train sequence:\n')
print(y_train)
x_train = torch.FloatTensor([x_train])
y_train = torch.FloatTensor(y_train)

X_train sequence:

[array([0. , 0.1, 0.2]), array([0.1, 0.2, 0.3]), array([0.2, 0.3, 0.4]), array([0.3, 0.4, 0.5]), array([0.4, 0.5, 0.6]), array([0.5, 0.6, 0.7]), array([0.6, 0.7, 0.8])]

Y_train sequence:

[1.6, 1.8, 2.0, 2.2, 2.4000000000000004, 2.6, 2.8]


In [30]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(Model, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.linear(out)
model = Model(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [41]:
for step in range(100):
    optimizer.zero_grad()
    y_pred = model(x_train)
    loss = torch.nn.functional.mse_loss(y_pred.flatten(), y_train)
    loss.backward()
    optimizer.step()
    if step % 10 == 0:
        print('iter:', step, ' loss:', loss.item())


iter: 0  loss: 0.014356245286762714
iter: 10  loss: 0.012688360176980495
iter: 20  loss: 0.011183018796145916
iter: 30  loss: 0.009839385747909546
iter: 40  loss: 0.008659387938678265
iter: 50  loss: 0.007645059376955032
iter: 60  loss: 0.006791248917579651
iter: 70  loss: 0.006088207475841045
iter: 80  loss: 0.0055214278399944305
iter: 90  loss: 0.005073525942862034
